In [ ]:
## import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.image as mpimg
import pandas as pd
from sklearn.cluster import KMeans


In [ ]:
data_path = 'Dataset/DatasetA/'

# create directory to store results
!mkdir Results
!mkdir Results/Q4
!mkdir Results/Q4/partA

pathA = "Results/Q4/partA/"

In [ ]:
# get name of images
import os
jpg_files = []
for root, dirs, files in os.walk("."):
    for filename in files:
        if "jpg" in filename:
            jpg_files.append(filename)

# Part A

In [ ]:
# function that divides a greyscale image into equally sized non-overlapping windows and returns the 
# feature descriptor for each window as distribution of LBP codes.

# input : image : image to take LBP of 
#         window_width : width of window to divide full image
#         window_length : length of window to divide full image
#.        border : how do we want to handel the border : 3 option , ignore border, mirror or pad with zeros
#         plot : boolean to plot image and save

# return 

def ICV_LBP(image,window_length,window_width,border_case,plot):
    
    # to store lbp of image
    lbp_full_image = np.zeros(image.shape)
    
    # to store lbps based on windows
    lbp_window = []
    
    # this will implement border_case == 'ignore_boreder' by starting and ending for loop accordingly
    ignore_int = 0
    
    # new image created for implementing different border halding methods
    new_image = np.zeros((image.shape[0]+2,image.shape[1]+2))
    new_image[1:image.shape[0]+1,1:image.shape[1]+1] = image.copy()
    
    # we have to take care of borders. We can either put zeros or take mirror image.
    # 3 cases of how to deal with borders
    if border_case == 'ignore_border':
        ignore_int = 1
        
    if border_case == 'mirror':
        # pad image with first and last row and columns
        new_image[0,1:image.shape[1]+1] = image[0,:]
        new_image[-1,1:image.shape[1]+1] = image[-1,:]
        new_image[1:image.shape[0]+1,0] = image[:,0]
        new_image[1:image.shape[0]+1,-1] = image[:,-1]
        
    if border_case == 'zeros':
        # pad image with first and last row and columns
        # already padded with zeros
        ignore_int = 0
        
    # consider a single window in every iteration of i,j loop
    for i in range(1,1+image.shape[0],window_width):
        for j in range(1,1+image.shape[1],window_length):
            
            # to store local binary descriptor for each pixel
            lbp_freq = np.zeros((256))
            # working on window in k,l loop
            for k in range(i+ignore_int,min(i+window_width,image.shape[0])-ignore_int):
                for l in range(j+ignore_int,min(j+window_length,image.shape[1])-ignore_int):
                    
                    # get bits and convert to decimal
                    bit_1 = new_image[k-1,l-1] > new_image[k,l]
                    bit_2 = new_image[k-1,l] > new_image[k,l]
                    bit_3 = new_image[k-1,l+1]>new_image[k,l]
                    bit_4 = new_image[k,l+1] > new_image[k,l]
                    bit_5 = new_image[k+1,l+1] > new_image[k,l]
                    bit_6 = new_image[k+1,l]> new_image[k,l]
                    bit_7 = new_image[k+1,l-1]> new_image[k,l]
                    bit_8 = new_image[k,l-1]> new_image[k,l]

                    decimal_number = bit_1*np.power(2,7) + bit_2*np.power(2,6) + bit_3*np.power(2,5)+ bit_4*np.power(2,4)+ bit_5*np.power(2,3)+ bit_6*np.power(2,2)+ bit_7*np.power(2,1)+ bit_8*np.power(2,0)
                    
                    # store lbp value for classification and printing later
                    lbp_full_image[k-1,l-1] = decimal_number
                    lbp_freq[decimal_number] = lbp_freq[decimal_number] + 1
            
            
            if(plot == True):
                # window image
                fig = plt.figure()
                sidx_x = i+ignore_int-1
                sidx_y = j+ignore_int-1
                eidx_x = min(i+window_width,image.shape[0])-ignore_int -1
                eidx_y = min(j+window_length,image.shape[1])-ignore_int - 1
                plt.imshow(image[sidx_x:eidx_x,sidx_y:eidx_y],cmap='gray')
                plt.title("image window for row and columns, "+str(i-1)+" , "+str(j-1))
                fig.savefig(pathA+"image_Frame_"+str(i-1)+" , "+str(j-1)+".png")
                plt.show()

                # lbp of window image
                fig = plt.figure()
                sidx_x = i+ignore_int-1
                sidx_y = j+ignore_int-1
                eidx_x = min(i+window_width,image.shape[0])-ignore_int -1
                eidx_y = min(j+window_length,image.shape[1])-ignore_int - 1
                # store window lbp 
                lbp_window.append(lbp_full_image[sidx_x:eidx_x,sidx_y:eidx_y])
                plt.imshow(lbp_full_image[sidx_x:eidx_x,sidx_y:eidx_y],cmap='gray')
                plt.title("LBP for window row and columns, "+str(i-1)+" , "+str(j-1))
                fig.savefig(pathA+"LBP_Frame_"+str(i-1)+" , "+str(j-1)+".png")
                plt.show()

                # normalize histogram
                lbp_freq = lbp_freq/np.sum(lbp_freq)
                # use pyplot.bar to make histogram
                fig = plt.figure()
                plt.bar(np.arange(lbp_freq.shape[0]),lbp_freq)
                plt.xlabel("LBP number in decimal")
                plt.ylabel("Number of pixels in bin")
                plt.title("LBP histogram for window row and columns, "+str(i-1)+" , "+str(j-1))
                fig.savefig(pathA+"Histogram_"+str(i-1)+" , "+str(j-1)+".png")
                plt.show()
            
    return lbp_full_image,lbp_window

            
            

In [ ]:
# load image to find LBP
figure(figsize=(14, 11))

# can change index from 5 to any number from 0 to 5 to get lbp of that image.

img = cv2.imread(data_path+jpg_files[5])

# take grayscale of image
img = img[:,:,0]*1/3 + img[:,:,1]*1/3 + img[:,:,2]*1/3
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
# get lbp of full image and windows using a window size of 128x128
lbp_full_image,lbp_window = ICV_LBP(img,128,128,"mirror",True)

In [ ]:
# lbp of full image
figure(figsize=(14, 11))
plt.imshow(lbp_full_image,cmap='gray')
plt.show()

### Part B

In [ ]:
# takes in lbp of full image and returns an array of lbps of all windows in an image
# input : image : LBP of full image
#         window_lenght : window length in which to divide full image lbp
#         window_width : window width in which to divide full image lbp
# return : array of lbps of windows
def ICV_lbp_descriptor(image,window_length,window_width):
    
    # create the full image lbp using mirror adjustment. for borders
    lbp_full_image,lbp_window = ICV_LBP(image,window_length,window_width,"mirror",False)
    
    # to store reshaped lbp of every window
    window_lbps = []
    # loop through the full image lbp
    for i in range(0,image.shape[0],window_width):
        for j in range(0,image.shape[1],window_length):
            
            # get lbp of window and store it
            lbp_window = lbp_full_image[i:i+window_width,j:j+window_length]
            window_lbps.append(lbp_window.reshape(-1,1))
            
    return np.squeeze(np.array(window_lbps))

In [ ]:
# loop through each image and get lbp descriptors using ICV_lbp_descriptor
def ICV_all_descriptor(filenames,window_length,window_width):
    global_lbp_descriptors = []
    for i in range(len(filenames)):
        print("working on image ",i)
        img = cv2.imread(data_path+filenames[i])
        # as opencv loads in BGR format by default, we want to show it in RGB.
        img = img[:,:,0]*1/3 + img[:,:,1]*1/3 +img[:,:,2]*1/3 
        window_lbps = ICV_lbp_descriptor(img,window_length,window_width)
        global_lbp_descriptors.append(window_lbps)
    return global_lbp_descriptors

###  Plotting concatenated global descriptors, this will not be used for classificaiton

In [ ]:
# get all descriptors for all images
all_lbp_descriptors = ICV_all_descriptor(jpg_files,128,128)


In [ ]:
for i in range(len(jpg_files)):
    print("working on image ",i)
    img = cv2.imread(data_path +jpg_files[i])
    img = img[:,:,0]*1/3 + img[:,:,1]*1/3 + img[:,:,2]*1/3
    print(jpg_files[i])
    plt.imshow(img,cmap='gray')
    glbp = all_lbp_descriptors[i]
    
    # this will store concatenated lbp
    lbp_freq = np.zeros((glbp.shape[0],256))
    # concatenate lbps from all windows
    for j in range(glbp.shape[0]):
        for k in range(glbp.shape[1]):
            lbp_freq[j,int(glbp[j][k])] = lbp_freq[j,int(glbp[j][k])] + 1
    
    lbp_freq = lbp_freq/np.sum(lbp_freq,1)[:,np.newaxis]
    # concatenate lbp
    lbp_freq = np.squeeze(lbp_freq.reshape(-1,1))
    fig = plt.figure(figsize=(10, 10), dpi=80)
    plt.bar(np.arange(lbp_freq.shape[0]),lbp_freq)
    plt.xlabel("LBP number in decimal")
    plt.ylabel("Number of pixels in bin")
    plt.title("LBP histogram for full image")
    plt.show()

##### Classification 

In [ ]:
# Given all local descriptors, compute the similarity between corresponding window lbps in all images.
# then similarity between two images is defines as the mean of corresponding window similarities
# Using this we can get similarity between all images to get a matrix num_images x num_images
# Apply k-means on this similarity matrix to get labels.

# input : all_lbp_descriptors : list of window lbps for all images
#         filenames : names of images 
#         return : similarity datafram

def ICV_lbp_classify(all_lbp_descriptors,filenames):
    
    # find out count of lbp for each window in every image. This will help in calculating histogram similarity
    lbp_count_all_image = []
    for i in range(len(all_lbp_descriptors)):
        lbp_count_image = np.zeros((all_lbp_descriptors[i].shape[0],256))
        for j in range(all_lbp_descriptors[i].shape[0]):
            temp_descriptor = all_lbp_descriptors[i][j,:]
            for k in range(temp_descriptor.shape[0]):
                lbp_count_image[j,int(temp_descriptor[k])] = lbp_count_image[j,int(temp_descriptor[k])] + 1
        lbp_count_image = lbp_count_image/np.sum(lbp_count_image,1)[:,np.newaxis]
        lbp_count_all_image.append(lbp_count_image)
        
    # compute histogram similarity of one image with every other image. Take mean of histogram similarity across window.

    similarity_matrix = np.zeros((len(filenames),len(filenames)))
    for i in range(len(lbp_count_all_image)):
        for j in range(len(lbp_count_all_image)):
            hist_similarity = 2*np.sum(np.minimum(lbp_count_all_image[i],lbp_count_all_image[j]),1)/(np.sum(lbp_count_all_image[i],1)+np.sum(lbp_count_all_image[j],1))
            mean_hist_similarity = np.mean(hist_similarity)
            similarity_matrix[i,j] = mean_hist_similarity

        
    similarity_df = pd.DataFrame(similarity_matrix, columns = jpg_files, index=jpg_files)
    display(similarity_df)
    
    # use k means to give labels
    kmeans = KMeans(n_clusters=2, random_state=0).fit(similarity_matrix)
    print("labels for images",kmeans.labels_)
    
    return similarity_df

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,128,128)
df_128 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

# Part C

c) Decrease the window size and perform classification again. Comment the results in the report.

In [ ]:
# Decrease the widow size from 128X128 to 4x4,8x8,16x16, 632x32 and 64x64. Store results to analyse later.

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,4,4)
df_4 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,8,8)
df_8 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,16,16)
df_16 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,32,32)
df_32 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,64,64)
df_64 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

# Part D

d) Increase the window size and perform classification again. Comment the results in the report.

In [ ]:
# Increasing window size from 128X128 to 256X256

In [ ]:
all_lbp_descriptors = ICV_all_descriptor(jpg_files,256,256)
for i in range(len(all_lbp_descriptors)):
    all_lbp_descriptors[i] = all_lbp_descriptors[i][:,np.newaxis].T
df_256 = ICV_lbp_classify(all_lbp_descriptors,jpg_files)

## Analysing effect of window size on classifier 

In [ ]:
# plot the mean of all car and face images and compare with means of off diagonal

In [ ]:
cars_mean_256 = (np.sum(np.array(df_256)[3:,3:])-3)/6
cars_mean_128 = (np.sum(np.array(df_128)[3:,3:])-3)/6
cars_mean_64 = (np.sum(np.array(df_64)[3:,3:])-3)/6
cars_mean_32 = (np.sum(np.array(df_32)[3:,3:])-3)/6
cars_mean_16 = (np.sum(np.array(df_16)[3:,3:])-3)/6
cars_mean_8 = (np.sum(np.array(df_8)[3:,3:])-3)/6
# cars_mean_2 = (np.sum(np.array(df_2)[3:,3:])-3)/6

face_mean_256 = (np.sum(np.array(df_256)[:3,:3])-3)/6
face_mean_128 = (np.sum(np.array(df_128)[:3,:3])-3)/6
face_mean_64 = (np.sum(np.array(df_64)[:3,:3])-3)/6
face_mean_32 = (np.sum(np.array(df_32)[:3,:3])-3)/6
face_mean_16 = (np.sum(np.array(df_16)[:3,:3])-3)/6
face_mean_8 = (np.sum(np.array(df_8)[:3,:3])-3)/6
# face_mean_2 = (np.sum(np.array(df_2)[:3,:3])-3)/6

In [ ]:
# off diagonal mean
off_mean_256 = np.mean(np.array(df_256)[3:,:3]+np.array(df_256)[:3,3:])/2
off_mean_128 = np.mean(np.array(df_128)[3:,:3]+np.array(df_128)[:3,3:])/2
off_mean_64 = np.mean(np.array(df_64)[3:,:3]+np.array(df_64)[:3,3:])/2
off_mean_32 = np.mean(np.array(df_32)[3:,:3]+np.array(df_32)[:3,3:])/2
off_mean_16 = np.mean(np.array(df_16)[3:,:3]+np.array(df_16)[:3,3:])/2
off_mean_8 = np.mean(np.array(df_8)[3:,:3]+np.array(df_8)[:3,3:])/2

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)
x_labels = ['8x8','16x16','32x32','64x64','128x128','256x256']
plt.plot(x_labels,[cars_mean_8,cars_mean_16,cars_mean_32,cars_mean_64,cars_mean_128,cars_mean_256],label='mean of similarity between cars')
plt.plot(x_labels,[face_mean_8,face_mean_16,face_mean_32,face_mean_64,face_mean_128,face_mean_256],label='mean of similarity between faces')
plt.plot(x_labels,[off_mean_8,off_mean_16,off_mean_32,off_mean_64,off_mean_128,off_mean_256],label='mean of similarity between heterogeneous labels')
plt.legend()
plt.title("effect of size of classification")
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)
x_labels = ['8x8','16x16','32x32','64x64','128x128','256x256']
car_mean_array = np.array([cars_mean_8,cars_mean_16,cars_mean_32,cars_mean_64,cars_mean_128,cars_mean_256])
face_mean_array = np.array([face_mean_8,face_mean_16,face_mean_32,face_mean_64,face_mean_128,face_mean_256])
off_mean_array = np.array([off_mean_8,off_mean_16,off_mean_32,off_mean_64,off_mean_128,off_mean_256])

spread1 = car_mean_array - off_mean_array
spread2 = face_mean_array - off_mean_array

plt.plot(x_labels,spread1,label='spread 1 : car_mean - heterogeneous_mean')
plt.plot(x_labels,spread2,label='spread 2 : face_mean - heterogeneous_mean')
plt.legend()
plt.title("spread between same type images and different type images")
plt.show()